<a href="https://colab.research.google.com/github/fgsantosti/ProgramacaoDispositivosMoveisFlutter/blob/main/App_Fluuter13_CRUD_GraphQL_Hasura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Todo esse conteúdo foi adaptado de acordo com a aula realizada pelo aluno `Cristian Paulo` que acoteceu em 2022 e está disponível no youtube do curso.

[**Link da aula**](https://www.youtube.com/watch?v=NmPmb6569H0&t=5674s)

Inicialmente iremos criar uma conta no site [Hasura](https://hasura.io/) iremos criar nosso banco para enviarmos informações via querys GraphQL API, de forma flexíveis e seguras sem se afogar na complexidade de microsserviços.

`Explicação na plataforma`

`pubspec.yaml`

```dart
dependencies:
  flutter:
    sdk: flutter
  hasura_connect:
  
```

Inciamente iremos criar uma pasta chamada de constantes, onde iremos criar o arquivo `constantes.dart`, neste iremos criar duas variáveis que iremos utilizar durante o processo de desenvovimento do app.

In [ ]:
final url = 'url-app';
final key = 'key';

Nosso aplicativo será um CRUD de carros onde iremos enviar três informações para o banco de dados:
* `id`
* `name`
* `decription`

`id` será auto-incremente no banco fizemos essa definição, `name` será o nome do carro, `description` será uma breve descrição do veículo.

Para isso iremos criar nossa pasta model onde teremos o arquivo `car_model.dart`

In [ ]:
class CarModel {
  Data? data;

  CarModel({this.data});

  CarModel.fromJson(Map<String, dynamic> json) {
    data = json['data'] != null ? new Data.fromJson(json['data']) : null;
  }

  Map<String, dynamic> toJson() {
    final Map<String, dynamic> data = new Map<String, dynamic>();
    if (this.data != null) {
      data['data'] = this.data!.toJson();
    }
    return data;
  }
}

class Data {
  List<Cars>? cars;

  Data({this.cars});

  Data.fromJson(Map<String, dynamic> json) {
    if (json['cars'] != null) {
      cars = <Cars>[];
      json['cars'].forEach((v) {
        cars!.add(new Cars.fromJson(v));
      });
    }
  }

  Map<String, dynamic> toJson() {
    final Map<String, dynamic> data = new Map<String, dynamic>();
    if (this.cars != null) {
      data['cars'] = this.cars!.map((v) => v.toJson()).toList();
    }
    return data;
  }
}

class Cars {
  int? id;
  String? name;
  String? description;

  Cars({this.id, this.name, this.description});

  Cars.fromJson(Map<String, dynamic> json) {
    id = json['id'];
    name = json['name'];
    description = json['description'];
  }

  Map<String, dynamic> toJson() {
    final Map<String, dynamic> data = new Map<String, dynamic>();
    data['id'] = this.id;
    data['name'] = this.name;
    data['description'] = this.description;
    return data;
  }
}


Iremos criar agora a pasta widgets onde iremos criar o arquivo `card_car.dart` nele temos um widget específico que será utilizado quando for realizada uma edição ou deleção de algum carro da lista.

In [ ]:
import 'package:flutter/material.dart';

class CardCar extends StatelessWidget {
  final String name;
  final String description;
  final Function()? onEdit;
  final Function()? onDelete;
  CardCar(
      {Key? key,
      required this.name,
      this.onEdit,
      this.onDelete,
      required this.description})
      : super(key: key);

  @override
  Widget build(BuildContext context) {
    return Container(
      width: 250,
      margin: const EdgeInsets.all(20),
      padding: const EdgeInsets.all(20),
      decoration: BoxDecoration(
        color: Colors.grey.shade200,
        borderRadius: BorderRadius.circular(20),
      ),
      child: Column(
        crossAxisAlignment: CrossAxisAlignment.stretch,
        children: [
          Text(name),
          Text(description),
          Row(
            mainAxisAlignment: MainAxisAlignment.end,
            children: [
              OutlinedButton(
                onPressed: onEdit,
                child: const Text('Editar'),
              ),
              TextButton(
                onPressed: onDelete,
                child: const Text('Deletar'),
                style: ButtonStyle(),
              ),
            ],
          ),
        ],
      ),
    );
  }
}


Logo depois disso vamos criar a pasta screens onde iremos criar um arquivo chamado `car_page.dart` será um formulário simples com dois `TextField` que irá pegar as informações `name` e `description`

In [ ]:
import 'package:flutter/material.dart';
import '../main.dart';
import '../model/car_model.dart';

class CarPage extends StatefulWidget {
  final Cars? car;
  const CarPage({super.key, this.car});

  @override
  State<CarPage> createState() => _CarPageState();
}

class _CarPageState extends State<CarPage> {
  var nameController = TextEditingController();
  var descricaoController = TextEditingController();

  @override
  void initState() {
    _init();
    super.initState();
  }

  Future<void> _init() async {
    if (widget.car != null) {
      nameController.text = widget.car!.name!;
      descricaoController.text = widget.car!.description!;
      setState(() {});
    }
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text("Adicione um Carro"),
      ),
      body: Column(
        mainAxisAlignment: MainAxisAlignment.center,
        children: [
          Expanded(
            child: Padding(
              padding: EdgeInsets.all(30),
              child: SizedBox(
                height: 250,
                child: TextField(
                  controller: nameController,
                  decoration: InputDecoration(
                      filled: true,
                      fillColor: Colors.grey.shade200,
                      hintText: "Digite..."),
                  expands: true,
                  maxLines: null,
                  onChanged: (value) {
                    setState(() {});
                  },
                ),
              ),
            ),
          ),
          Expanded(
            child: Padding(
              padding: EdgeInsets.all(30),
              child: SizedBox(
                height: 250,
                child: TextField(
                  controller: descricaoController,
                  decoration: InputDecoration(
                      filled: true,
                      fillColor: Colors.grey.shade200,
                      hintText: "Digite..."),
                  expands: true,
                  maxLines: null,
                  onChanged: (value) {
                    setState(() {});
                  },
                ),
              ),
            ),
          ),
          Expanded(
            child: Padding(
              padding: EdgeInsets.all(30),
              child: SizedBox(
                height: double.infinity,
                child: ElevatedButton.icon(
                  onPressed: nameController.text == ""
                      ? null
                      : () async {
                          late bool isSend;
                          if (widget.car != null) {
                            isSend = await update(widget.car!.id!,
                                nameController.text, descricaoController.text);
                          } else {
                            isSend = await sendCar(
                                nameController.text, descricaoController.text);
                          }
                          if (isSend) {
                            Navigator.pop(context, isSend);
                          }
                        },
                  icon: const Icon(
                    Icons.send,
                  ),
                  label: const Text(
                    'Adicionar carro',
                  ),
                ),
              ),
            ),
          )
        ],
      ),
    );
  }

  Future<bool> sendCar(String name, String description) async {
    String documentMutation = """
      mutation MyMutation {
        insert_cars(objects: {name: "$name", description: "$description" }) {
          affected_rows
        }
      }
    """;

    try {
      await hasuraConnect.mutation(documentMutation);
      return true;
    } catch (e) {
      return false;
    }
  }

  Future<bool> update(int id, String name, String description) async {
    String document = """
      mutation MyMutation {
        update_cars(where: {id: {_eq: $id }}, _set: {name: "$name", description: "$description"}) {
          affected_rows
        }
      }
    """;
    try {
      await hasuraConnect.mutation(document);
      return true;
    } catch (e) {
      return false;
    }
  }
}


Por fim iremos implementar o arquivo main.dart, onde iremos juntar tudo, nela irá ser listadas todos os carros existente no banco e onde teremos um botão para inserir novos carros.

In [ ]:
import 'constantes/constantes.dart';
import 'model/car_model.dart';
import 'sreens/car_page.dart';
import 'widgets/card_car.dart';
import 'package:flutter/cupertino.dart';
import 'package:flutter/material.dart';
import 'package:hasura_connect/hasura_connect.dart';


final HasuraConnect hasuraConnect = HasuraConnect(
  url,
  headers: {'x-hasura-admin-secret': key},
);

void main() {
  runApp(const MyApp());
}

class MyApp extends StatelessWidget {
  const MyApp({Key? key}) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Flutter Demo',
      theme: ThemeData(
        primarySwatch: Colors.blue,
        useMaterial3: true,
      ),
      home: MyHomePage(),
      routes: {
        "car_page": (context) {
          var result = ModalRoute.of(context)?.settings.arguments;
          Cars? car = result == null ? null : result as Cars;
          return CarPage(
            car: car,
          );
        }
      },
    );
  }
}

class MyHomePage extends StatefulWidget {
  const MyHomePage({Key? key}) : super(key: key);

  @override
  State<MyHomePage> createState() => _MyHomePageState();
}

class _MyHomePageState extends State<MyHomePage> {
  String queryCar = """
    query MyQuery {
      cars(order_by: {id: asc})  {
        id
        name
        description
      }
    }
  """;

  Future<CarModel> getAllCar() async {
    var result = await hasuraConnect.query(queryCar);
    return CarModel.fromJson(result);
  }

  Future<bool> delete(int id) async {
    String document = """
      mutation MyMutation {
        delete_cars(where: {id: {_eq: $id }}) {
          affected_rows
        }
      }
    """;

    try {
      await hasuraConnect.mutation(document);
      return true;
    } catch (e) {
      return false;
    }
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text("Revendendora de Carros"),
      ),
      body: FutureBuilder<CarModel>(
          future: getAllCar(),
          builder: (context, snapshot) {
            if (!snapshot.hasData) {
              return const Center(child: CircularProgressIndicator());
            }
            return ListView.builder(
              itemCount: snapshot.data!.data!.cars!.length,
              itemBuilder: (context, i) {
                var cars = snapshot.data!.data!.cars!.elementAt(i);
                return CardCar(
                  name: cars.name!,
                  onEdit: () async {
                    await Navigator.pushNamed(
                      context,
                      "car_page",
                      arguments: cars,
                    );
                    setState(() {});
                  },
                  onDelete: () async {
                    await delete(cars.id!);
                    setState(() {});
                  },
                  description: cars.description!,
                );
              },
            );
          }),
      floatingActionButton: FloatingActionButton.extended(
        label: const Text("Adicinar"),
        icon: const Icon(CupertinoIcons.add),
        onPressed: () async {
          var isSend = await Navigator.pushNamed(context, "car_page");
          if (isSend != null) {
            setState(() {});
          }
        },
      ),
    );
  }
}
